# 1

回溯法，能到达0即可加入结果。
为减少查询，从大到小开始。

In [15]:
from typing import List

class Solution:
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        ans = []
        for k in range(len(candidates)-1, -1, -1):
            pos = k
            record = []
            tmp = target
            while tmp > 0 and pos >= 0:
                tmp = tmp - candidates[pos]
                # print(tmp)
                if tmp > 0:
                    record.append(candidates[pos])
                elif tmp < 0:
                    pos -= 1
                elif tmp == 0:
                    record.append(candidates[pos])
                    ans.append(record)
                    break
        return ans


if __name__ == '__main__':
    candidates_list = [
        [2,3,6,7],
        [2,3,5],
        [2],
        [1],
        [1],
    ]
    target_list = [
        7,
        8,
        1,
        1,
        2,
    ]

    for candidates, target in zip(candidates_list, target_list):
        print(Solution().combinationSum(candidates, target))

[[7]]
[[2, 2, 2, 2]]
[]
[[1]]
[[1, 1]]


没过！考虑的格局不够。总想从局部条件去一一判断。

# 题解

搜索回溯，
每个元素的选择有2种，要么用当前元素；要么不用当前元素，并进入下一个元素的抉择。
因为元素可重复，所以，用了当前元素后，还可以抉择是否继续使用当前元素，或者进入下一元素的抉择。

宏观上，形成了二叉树的形式：
```
                        7
                    [2,3,6,7]
                    /       \
                next      use 2
                /               \
              7                  5
        [_,3,6,7]            [2,3,6,7]
         /    \                  /    \
       next   use 3           next   use 2
      /          \             /         \
     7            4           5           3
[_,_,6,7]     [_,_,6,7]    [_,3,6,7]   [2,3,6,7]
 ...             ...        ...           ...
```
  
对二叉树的搜索（dfs, bfs），加上剪枝可以优化时间。（排序，在后面元素都不可能时，直接剪掉。）

（本质看上去就是 枚举。不过考虑清除了如何枚举方式，不重复。）

In [14]:
# from typing import *

# class Solution:
#     def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
#         self.ans = []
#         self.dfs(candidates, target, 0, [])
#         return self.ans

#     def dfs(self, candidates, target, idx, record):
#         if idx >= len(candidates) or target <= 0:
#             if target == 0:
#                 self.ans.append(record)
#             return

#         tmp = record.copy()                                     # 跳过该元素，为了让record各自记录其分支情况
#         self.dfs(candidates, target, idx + 1, tmp)              # 跳过该元素

#         tmp = record.copy()                                     # 使用当前元素， 为了让record各自记录其分支情况
#         tmp.append(candidates[idx])                             # 使用当前元素
#         self.dfs(candidates, target-candidates[idx], idx, tmp)  # 使用当前元素


# if __name__ == '__main__':
#     candidates_list = [
#         [2,3,6,7],
#         [2,3,5],
#         [2],
#         [1],
#         [1],
#     ]
#     target_list = [
#         7,
#         8,
#         1,
#         1,
#         2,
#     ]

#     for candidates, target in zip(candidates_list, target_list):
#         print(Solution().combinationSum(candidates, target))

执行用时：116 ms, 在所有 Python3 提交中击败了14.47%的用户

内存消耗：15.1 MB, 在所有 Python3 提交中击败了23.43%的用户

优化记录组合的变量record，通过push和pop成对，避免每次dfs分支都复制一份。

In [13]:
from typing import *

class Solution:
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        self.ans = []
        self.dfs(candidates, target, 0, [])
        return self.ans

    def dfs(self, candidates, target, idx, record):
        if idx == len(candidates):
            return
        if target == 0:
            self.ans.append(record.copy())                              # 防止record被接下来的操作覆盖
            return
        
        self.dfs(candidates, target, idx + 1, record)                   # 跳过该元素

        if target - candidates[idx] >= 0:
            record.append(candidates[idx])                              # push当前元素
            self.dfs(candidates, target-candidates[idx], idx, record)   # 使用当前元素
            record.pop()                                                # pop"最后"元素（递归的最里层依次返回，pop的最后其实就是当前元素）


if __name__ == '__main__':
    candidates_list = [
        [2,3,6,7],
        [2,3,5],
        [2],
        [1],
        [1],
    ]
    target_list = [
        7,
        8,
        1,
        1,
        2,
    ]

    for candidates, target in zip(candidates_list, target_list):
        print(Solution().combinationSum(candidates, target))

[[7], [2, 2, 3]]
[[3, 5], [2, 3, 3], [2, 2, 2, 2]]
[]
[[1]]
[[1, 1]]


执行用时：64 ms, 在所有 Python3 提交中击败了59.64%的用户

内存消耗：15 MB, 在所有 Python3 提交中击败了70.05%的用户

time:O(2^n * n) 最差2^n种组合都会被枚举到，O(n)为列表复制。

space:O(target) 最差需要递归O(target)层。

注：record.append()返回值为None，将其作为参数传入，会出现record为None的错误。
```python
        self.dfs(candidates, target-candidates[idx], idx, record.copy().append(candidates[idx]))   # 使用当前元素
```